### GridSearch & Pipelines
GridSearch is an optimization tool that we use when tuning hyperparameters. We define the grid of parameters that we want to search through, and we select the best combination of parameters for our data.

# 1 - One way
Itera un algoritmo sobre un conjunto de hiperparametros

In [1]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [3]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV, train_test_split

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore",category=DeprecationWarning)
warnings.filterwarnings(action="ignore",category=FutureWarning)

iris = datasets.load_iris()

X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size = 0.2,
                                                   random_state=42)

svc = SVC()

parameters = {
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'C': [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 100],
    'gamma': ['scale', 'auto'],
    'coef0': [-1, 0, 1]
}

grid = GridSearchCV(estimator = svc,
                   param_grid = parameters,
                   n_jobs = -1,
                   scoring = 'accuracy',
                   cv = 10)
grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 100],
                         'coef0': [-1, 0, 1], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='accuracy')

In [4]:
print("Best estimator:", grid.best_estimator_)
print("Best params:", grid.best_params_)
print("Best score:", grid.best_score_)

Best estimator: SVC(C=0.1, coef0=1, kernel='poly')
Best params: {'C': 0.1, 'coef0': 1, 'gamma': 'scale', 'kernel': 'poly'}
Best score: 0.9666666666666666


In [5]:
best_estimator = grid.best_estimator_
best_estimator.score(X_test, y_test)

1.0

# 2: Almost-Pro way

La forma pro es la que hace esto mismo y va recogiendo los errores de entrenamiento, de validación y tiene la capacidad de parar el proceso cuando se requiera además de guardar el modelo en local una vez terminado si es mejor que el que había anteriormente y de cargar el modelo anterior y seguir reentrenando.

In [6]:
pipe = Pipeline(steps=[('classifier', RandomForestClassifier())])

logistic_params = {
    'classifier': [LogisticRegression()],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__C': np.arange(0.1, 4, 0.5)
}

random_forest_params = {
    'classifier': [RandomForestClassifier()],
    'classifier__n_estimators': [10, 100, 500, 1000],
    'classifier__max_features': [1,2,3]
}

svc_params = {
    'classifier': [SVC()],
    'classifier__kernel': ['linear', 'rbf', 'sigmoid']
}

search_space = [logistic_params, random_forest_params, svc_params]

grid = GridSearchCV(pipe,
                   search_space,
                   cv=10,
                   n_jobs=-1)

grid.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('classifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid=[{'classifier': [LogisticRegression(C=1.6)],
                          'classifier__C': array([0.1, 0.6, 1.1, 1.6, 2.1, 2.6, 3.1, 3.6]),
                          'classifier__penalty': ['l1', 'l2']},
                         {'classifier': [RandomForestClassifier()],
                          'classifier__max_features': [1, 2, 3],
                          'classifier__n_estimators': [10, 100, 500, 1000]},
                         {'classifier': [SVC()],
                          'classifier__kernel': ['linear', 'rbf', 'sigmoid']}])

In [7]:
grid.score(X_test, y_test)

1.0

In [8]:
print(grid.predict(X_test))
print(y_test)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


In [10]:
grid.best_estimator_['classifier']

LogisticRegression(C=1.6)

In [11]:
grid.best_score_

0.9583333333333334

# 3 Another way

In [15]:
reg_log = Pipeline(steps = [
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('reglog', LogisticRegression())
])

svc = Pipeline([
    ('scaler', StandardScaler()),
    ('selectkbest', SelectKBest()),
    ('svc', SVC())
])

rand_forest = RandomForestClassifier()

reg_log_param = {
    "imputer__strategy": ['mean', 'median', 'most_frequent'],
    "reglog__penalty": ["l1", "l2"],
    "reglog__C": np.arange(0.1, 4, 0.5)
}

svc_param = {
    "selectkbest__k": [1, 2, 3],
    "svc__C": np.arange(0.1, 0.9, 0.1),
    "svc__kernel": ['linear', 'poly', 'rbf']
}

rand_forest_params = {
    'n_estimators': [10, 100, 500, 1000],
    'max_features': [1, 2, 3]
}

gs_reg_log = GridSearchCV(reg_log,
                         reg_log_param,
                         cv = 10,
                         scoring = 'accuracy',
                         n_jobs = -1,
                         verbose = 1)

gs_svm = GridSearchCV(svc,
                         svc_param,
                         cv = 10,
                         scoring = 'accuracy',
                         n_jobs = -1,
                         verbose = 1)


gs_rand_forest = GridSearchCV(rand_forest,
                         rand_forest_params,
                         cv = 10,
                         scoring = 'accuracy',
                         n_jobs = -1,
                         verbose = 1)

grids = {
    "gs_reg_log": gs_reg_log,
    "gs_svm": gs_svm,
    "gs_rand_forest": gs_rand_forest
}

In [16]:
%%time
for nombre, grid_search in grids.items():
    grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:    8.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 10 folds for each of 72 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:    2.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  51 tasks      | elapsed:   16.4s


Wall time: 54.8 s


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   43.4s finished


In [17]:
grids.items()

dict_items([('gs_reg_log', GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('reglog', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'imputer__strategy': ['mean', 'median',
                                               'most_frequent'],
                         'reglog__C': array([0.1, 0.6, 1.1, 1.6, 2.1, 2.6, 3.1, 3.6]),
                         'reglog__penalty': ['l1', 'l2']},
             scoring='accuracy', verbose=1)), ('gs_svm', GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selectkbest', SelectKBest()),
                                       ('svc', SVC())]),
             n_jobs=-1,
             param_grid={'selectkbest__k': [1, 2, 3],
                         'svc__C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]),
    

In [19]:
best_grids = [(i, j.best_score_) for i, j in grids.items()]

best_grids = pd.DataFrame(best_grids, columns = ['Grid', 'Best score'])
best_grids.sort_values(by='Best score', ascending=False)

,Grid,Best score
1,gs_svm,0.950000
0,gs_reg_log,0.941667
2,gs_rand_forest,0.933333


In [20]:
print("Best estimator:", gs_svm.best_estimator_)
print("Best params:", gs_svm.best_params_)
print("Best score:", gs_svm.best_score_)

Best estimator: Pipeline(steps=[('scaler', StandardScaler()), ('selectkbest', SelectKBest(k=2)),
                ('svc', SVC(C=0.1, kernel='linear'))])
Best params: {'selectkbest__k': 2, 'svc__C': 0.1, 'svc__kernel': 'linear'}
Best score: 0.9499999999999998


In [22]:
estimador = gs_svm.best_estimator_

import pickle

with open('finished_model.model', 'wb') as archivo_salida:
    pickle.dump(estimador, archivo_salida)

In [23]:
with open('finished_model.model', 'rb') as archivo_entrada:
    pipeline_importado = pickle.load(archivo_entrada)

In [24]:
pipeline_importado

Pipeline(steps=[('scaler', StandardScaler()), ('selectkbest', SelectKBest(k=2)),
                ('svc', SVC(C=0.1, kernel='linear'))])